# Battle of The Neighborhoods

In this notebook I will be fetching location data using an API and using the data I will build a model to successfully perform clustering of neighborhoods with which are similar to eachother.

In [9]:
import pandas as pd
import numpy as np
print("Hello capstone project course!")

Hello capstone project course!


# 1) Webscraping and extracting neighborhood data
Import required libraries.

In [10]:
from bs4 import BeautifulSoup
import requests
import csv

In [11]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

Parsing the required table from the website and converting it into a pandas dataframe.

In [12]:
stuff = soup.find('div', class_='mw-parser-output').table
data = pd.DataFrame()
rowi = 0
for row in stuff.findAll('tr'):
    coli = 0
    for col in row.findAll('td'):
        data.loc[rowi,coli] = col.get_text()
        coli += 1
    rowi += 1

data.columns = ['PostCode', 'Borough', 'Neighborhood']
data['Neighborhood'] = pd.Series(data['Neighborhood']).str.rstrip('\n')

Here I am removing data values where Borough has not been assgined.

In [ ]:
data.drop(data[data['Borough'] == 'Not assigned'].index, inplace = True)
data.reset_index(drop = True, inplace = True)

Grouping Neighborhoods with same postal code and replaceing neighborhoods with borough values where it has not been assigned.

In [ ]:
df = data.groupby(['PostCode'], as_index = False).agg({'Borough' : 'max', 'Neighborhood': ','.join}, inplace = True)
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])
df.head()

,PostCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [ ]:
df.shape

(103, 3)

In [ ]:
!pip install geocoder
import geocoder
postal_code1 = df['PostCode']
long = []
lat = []
for code in postal_code1:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng
    long.append(lat_lng_coords[1])
    lat.append(lat_lng_coords[0])

     |████████████████████████████████| 102kB 16.5MB/s ta 0:00:01


In [ ]:
long_series = pd.Series(long)
lat_series = pd.Series(lat)
frame = {'PostCode': postal_code1 , 'Latitude': lat_series, 'Longitude': long_series}
df1 = pd.DataFrame(frame)
result = pd.merge(df,df1, on = 'PostCode' )
result.head()

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


### Getting coordinates for Toronto, CA 

In [ ]:
from geopy.geocoders import Nominatim
address = 'Toronto, CA'
geolocator = Nominatim(user_agent = 'ca_agent')
ngb_latlong = geolocator.geocode(address)
ngb_lat = ngb_latlong.latitude
ngb_long = ngb_latlong.longitude

### Creating a map of toronto using the folium library

In [ ]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: / 

## Plotting neighborhoods on the map of Toronto

In [ ]:
toronto_map =  folium.Map(location = [ngb_lat, ngb_long], zoom_start = 11)
for lat,lng,borough,neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat,lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(toronto_map)  
toronto_map

Here I am filtering out the bouroughs not containing 'Toronto'

In [ ]:
filtered_df = result[result['Borough'].str.contains('Toronto')].reset_index(drop = True)

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'XGTTY3F1R2SG4JQ4BQNN5KUQK0HNKOUWRKSYPGO3Q3OMW1DI' # your Foursquare ID
CLIENT_SECRET = 'B4ZG32CPPGC4M1GBMC50XWY1W3WS2GRUMMOQ2NZMYZQRWFVI' # your Foursquare Secret
VERSION = '20200131' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

### Writing a function to get nearby venues of all neighbhorhoods

In [ ]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=filtered_df['Neighborhood'],
                                   latitudes=filtered_df['Latitude'],
                                   longitudes=filtered_df['Longitude']
                                  )

Let's check how many venues were returned for each neighborhood

In [ ]:
toronto_venues.groupby('Neighborhood').count()

#### Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

## 3. Analyze Each Neighborhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

#### Let's print each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

### Put the top 10 venues of each neighbhorhood into a dataframe

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

## 4. Cluster Neighborhoods and Visualize the clusters on Toronto's map

In [ ]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = filtered_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

In [ ]:
toronto_merged['Cluster Labels'].dropna(inplace = True)

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[ngb_lat, ngb_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters